In [6]:
import pandas as pd
import numpy as np
import sqlalchemy

from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

base = declarative_base()

In [7]:
#Use Pandas to read your cleaned measurements and stations CSV data.
clean_station_df = pd.read_csv('clean_hawaii_stations.csv')
clean_measures_df = pd.read_csv('clean_hawaii_measurements.csv')

In [8]:
#Use the engine and connection string to create a database called hawaii.sqlite
engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()

In [9]:
base = declarative_base()

class Station(base):
    __tablename__='station'
    station = Column(Text, primary_key = True)
    name = Column(Text)
    latitude = Column(Float)
    longtitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}"

In [10]:
Station.__table__

Table('station', MetaData(bind=None), Column('station', Text(), table=<station>, primary_key=True, nullable=False), Column('name', Text(), table=<station>), Column('latitude', Float(), table=<station>), Column('longtitude', Float(), table=<station>), Column('elevation', Float(), table=<station>), schema=None)

In [11]:
class measurement(base):
    __tablename__='measurement'    
    station = Column(Text, primary_key = True)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}"

In [12]:
measurement.__table__

Table('measurement', MetaData(bind=None), Column('station', Text(), table=<measurement>, primary_key=True, nullable=False), Column('date', Date(), table=<measurement>), Column('prcp', Float(), table=<measurement>), Column('tobs', Float(), table=<measurement>), schema=None)

In [13]:
base.metadata.create_all(engine)

In [14]:
# First, let's examine the tables in the db# First,  
tables = engine.execute("SELECT * FROM sqlite_master WHERE TYPE='table'")
for index, table in enumerate(tables):
    table_name = table[1]
    print(f"The #{index} table in the database is called '{table_name}'.")

The #0 table in the database is called 'clean_hawaii_stations'.
The #1 table in the database is called 'station'.
The #2 table in the database is called 'clean_hawaii_measurements'.
The #3 table in the database is called 'measurement'.


In [15]:
base.metadata.create_all(engine)

In [16]:
session = Session(bind=engine)

In [17]:
df1 = clean_measures_df
df2 = clean_station_df

In [18]:
clean_measurements = pd.read_csv('clean_hawaii_measurements.csv')
clean_station = pd.read_csv('clean_hawaii_stations.csv')

In [19]:
weather_data = clean_measurements.to_dict(orient='records')
station_data = clean_station.to_dict(orient='records')
weather_data[0]

{'Unnamed: 0': 0,
 'date': '2010-01-01',
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [20]:
metadata1 = MetaData(bind=engine)
metadata1.reflect()

metadata2 = MetaData(bind=engine)
metadata2.reflect()

In [21]:
table1 = sqlalchemy.Table('measurement', metadata1, autoload=True)
table2 = sqlalchemy.Table('station', metadata2, autoload=True)

In [27]:
conn.execute(table1.insert(), weather_data)
conn.execute(table2.insert(), station_data)

StatementError: (builtins.TypeError) SQLite Date type only accepts Python date objects as input. [SQL: 'INSERT INTO measurement (station, date, prcp, tobs) VALUES (?, ?, ?, ?)'] [parameters: [{'Unnamed: 0': 0, 'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'Unnamed: 0': 1, 'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'Unnamed: 0': 2, 'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'Unnamed: 0': 3, 'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'Unnamed: 0': 4, 'station': 'USC00519397', 'date': '2010-01-06', 'prcp': 0.0, 'tobs': 73}, {'Unnamed: 0': 5, 'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}, {'Unnamed: 0': 6, 'station': 'USC00519397', 'date': '2010-01-08', 'prcp': 0.0, 'tobs': 64}, {'Unnamed: 0': 7, 'station': 'USC00519397', 'date': '2010-01-09', 'prcp': 0.0, 'tobs': 68}  ... displaying 10 of 19550 total bound parameter sets ...  {'Unnamed: 

In [26]:
session.commit()